In [1]:
import cv2
import numpy as np
import gradio as gr
from PIL import Image

# Fungsi untuk meningkatkan kualitas gambar
def improve_image(uploaded_image, sharpness_factor, blur_level, use_clahe):
    if uploaded_image is None:
        return None, None

    img = np.array(uploaded_image)
    
    # Konversi ke LAB untuk mempertahankan warna saat equalization
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Histogram Equalization atau CLAHE
    if use_clahe:
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
    else:
        l = cv2.equalizeHist(l)

    # Gabungkan kembali LAB dan konversi ke RGB
    lab_equalized = cv2.merge((l, a, b))
    img_enhanced = cv2.cvtColor(lab_equalized, cv2.COLOR_LAB2RGB)

    # Noise Reduction (Gaussian Blur)
    blur_size = (blur_level * 2 + 1, blur_level * 2 + 1)
    img_blur = cv2.GaussianBlur(img_enhanced, blur_size, 0)

    # Sharpening menggunakan Unsharp Masking
    gaussian = cv2.GaussianBlur(img_blur, (0, 0), sigmaX=2)
    sharpened = cv2.addWeighted(img_blur, 1.0 + sharpness_factor / 10, gaussian, -sharpness_factor / 10, 0)

    # Konversi kembali ke PIL
    result_img = Image.fromarray(sharpened)

    return uploaded_image, result_img

# Antarmuka Gradio
iface = gr.Interface(
    fn=improve_image,
    inputs=[
        gr.Image(type="pil"),
        gr.Slider(1, 10, value=5, label="Sharpness Factor"),
        gr.Slider(1, 5, value=2, label="Blur Level"),
        gr.Checkbox(label="Gunakan CLAHE (Peningkatan Kontras Adaptif)"),
    ],
    outputs=[gr.Image(type="pil", label="Sebelum"), gr.Image(type="pil", label="Sesudah")],
    title="Peningkatan Kualitas Gambar",
    description="Unggah gambar dan sesuaikan pengaturan untuk melihat hasil peningkatan kualitas.",
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
